In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pathlib as plb
import seaborn as sns
from scipy import stats
import statistics
import dabest as db

In [2]:
base_folder = plb.Path('C:/Users/Emily/Documents/LR')
md_path = base_folder.joinpath('CD3.csv')
md_df = pd.read_csv(md_path)
md_df['Scanner_Slot'] = md_df['Scanner_Slot'].fillna(0).astype(int)
md_df['Folder', 'Sub_Folder', ] = md_df['Scanner_Slot'].fillna(0).astype(int)
md_df.head()

,Date,Folder,Sub_Folder,GUI_Analysis_and_Image_Folder,Image_ID,Scanner_Slot,Plate_ID,Well_ID,Gel_Matrix,Compound_Vendor,Solvent_or_Reference,Dilution,Compound_Volume,Strain,Total_Worms,Chemotaxis_Index,"(Folder, Sub_Folder)"
0,20211205.0,2021_Month12_December,211205_Analysis,LRH_211205_001,LRH_211205_001_POST,1,1.0,A,Gelrite,Water,Water,Pure,2.5 ul,CX32,99.0,-0.315068,1
1,20211205.0,2021_Month12_December,211205_Analysis,LRH_211205_001,LRH_211205_001_POST,1,1.0,B,Gelrite,Water,Water,Pure,2.5 ul,GN1067,196.0,-0.479769,1
2,20211205.0,2021_Month12_December,211205_Analysis,LRH_211205_001,LRH_211205_001_POST,1,1.0,C,Gelrite,Water,Water,Pure,2.5 ul,VC2421,282.0,-0.053571,1
3,20211205.0,2021_Month12_December,211205_Analysis,LRH_211205_001,LRH_211205_001_POST,1,1.0,D,Gelrite,Water,Water,Pure,2.5 ul,VC4210,352.0,0.094203,1
4,20211205.0,2021_Month12_December,211205_Analysis,LRH_211205_001,LRH_211205_001_POST,2,2.0,A,Gelrite,Water,Water,Pure,2.5 ul,N2,280.0,-0.207843,2


In [12]:
strains = ['N2', 'CX32', 'VC2421', 'GN1067', 'VC4210', 'GN1066', 'GN1068']
compounds = ['Water']

In [64]:
def get_worm_locs(row, wrms, result_dict): 
    sub_folder = row['Sub_Folder']
    folder = row['GUI_Analysis_and_Image_Folder']
    filename = row['Image_ID']
    slot = str(row['Scanner_Slot'])
    well = row['Well_ID']
#loc_LRH_211214_001_POST_1A
    sub = wrms.joinpath(sub_folder)
    ssub =  sub.joinpath(folder)
    filepath = ssub.joinpath('loc_' + filename + '_' + slot + well + '.csv')
    #temp = pd.read_csv(loc_fname)
    
    
    if filepath.is_file():
        data = pd.read_csv(filepath)
        strain = row['Strain']
        xs = data['X']
    #xs = list(temp['centroid-1'])
        if strain in result_dict:
            result_dict[strain] = result_dict[strain].append(xs)
            result_dict[strain].reset_index(inplace=True, drop=True)
            result_dict[strain] = result_dict[strain]+xs
        else:
            result_dict[strain]=xs
        return result_dict
    
    else: 
        pass
    

In [26]:
def subst(df, strain, compound, path):
    temp = df.loc[(df['Strain']==strain) & 
            (df['Compound_Vendor']==compound)]
    # Need to create an empty dictionary to hold the values
    return temp


In [62]:
# 1 inch = 25.4mm
mm = 25.4
# 1200 pixels per 25.4mm
px_mm = 1200/mm

In [63]:
all_strains = pd.DataFrame()
compound = 'Water'
for s in strains:
    test = subst(md_df, s, compound, base_folder)
    all_strains = all_strains.append(test)
    
# Need to create an empty dictionary to hold the values
results_dict = {}
for index, row in all_strains.iterrows():
    pooled = get_worm_locs(row, base_folder, results_dict)

pooled_df = pd.DataFrame.from_dict(pooled)
mm_df = pooled_df.apply(lambda x: -(x/px_mm)+32.5)
pooled_df.to_csv(base_folder.joinpath(compound+'_dpi.csv'))
mm_df.to_csv(base_folder.joinpath(compound+'_mm.csv'))